In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip3 install wordcloud -q
from wordcloud import WordCloud

In [ ]:
df = pd.read_csv('output.csv', delimiter='|')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
for col in df:
    print(df[col].nunique())

In [ ]:
#Setting up data types
df['Activity'] = df['Activity'].astype('category')
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df['Recipient'] = df['Recipient'].astype('category')
df['Account'] = df['Account'].astype('category')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
df['Status'] = df['Status'].astype('category')

In [ ]:
total_spent = df[df['Activity'] == 'Paid']['Amount'].sum()

In [ ]:
total_received = df[df['Activity'] == 'Received']['Amount'].sum()

In [ ]:
print("Net Spendings: ",(total_spent-total_received))

In [ ]:
transactions_by_activity = df['Activity'].value_counts()

In [ ]:
filtered_df = df[(df['Activity'] == 'Paid') & (df['Amount'] > 100)]
grouped_df = filtered_df.groupby('Recipient')['Amount'].sum()
top_recipients = grouped_df[grouped_df > 100].sort_values(ascending=False)

In [ ]:
print(f"Total amount spent: {total_spent}")

In [ ]:
print(f"Total amount received: {total_received}")

In [ ]:
print(f"Transactions by activity:\n{transactions_by_activity}")

In [ ]:
print(f"Top 5 recipients by amount paid:\n{top_recipients.head()}")

In [ ]:
# Pie chart of transactions by activity type
plt.figure(figsize=(8, 6))
transactions_by_activity.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Transactions by Activity Type')
plt.ylabel('')
plt.show()

In [ ]:
# Bar chart of top recipients by amount paid
plt.figure(figsize=(10, 6))
top_recipients.plot(kind='bar')
plt.title('Top Recipients by Amount Paid')
plt.xlabel('Recipient')
plt.ylabel('Total Amount Paid')
plt.show()

In [ ]:
filtered_df = df[df['Activity'] == 'Paid']

# Group by 'Recipient' and sum the 'Amount'
grouped_df = filtered_df.groupby('Recipient')['Amount'].sum()

# Convert the grouped data to a dictionary
recipient_amount_dict = grouped_df.to_dict()

# Create a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(top_recipients)

# Plotting the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Recipients Based on Amount Paid')
plt.show()